In [1]:
%load_ext autoreload 
%autoreload 2

from pathlib import Path 

import subprocess



# TODOs

- if no `katan` in config, prompt user and write to SSH_CONFIG 
- option to use `-F configfile` option for ssh if different config file used by user. 

- allow client-side use of remote variables 
(e.g. typing $SCRATCH as the 'remote directory' will be evaluated on the HOST, and then parsed by the following `code` command 
to being whatever $SCRATCH actually is on the remote (arbitrary))


- remove requirement to have `*-*` hostname in `~/.ssh/config` before using the script 
(i.e. the script should be able to execute the SSH connection regardless)



In [ ]:
SSH_CONFIG_PATH = Path.home().resolve() / ".ssh" / "config"
assert SSH_CONFIG_PATH.is_file(), f"SSH config file not found at {SSH_CONFIG_PATH}"



True

In [4]:
# Get hostnames from ssh config
with open(SSH_CONFIG_PATH, "r") as f:
    ssh_config = f.read()

hostnames = []
for line in ssh_config.split("\n"):
    if line.startswith("Host "):
        hostnames.append(line.split()[1])
    
print(f"Found {len(hostnames)} hostnames in ssh config")


Found 7 hostnames in ssh config


In [10]:
KATANA = "katana"
KATANA_ADDR = "katana.restech.unsw.edu.au" 
USER = "USERNAME_GOES_HERE" # TODO: prompt 
if KATANA in hostnames:
    print(f"Found {KATANA} in ssh config")
else:
    print(f"{KATANA} not found in ssh config")
    
    appended = f"""
Host {KATANA}
    HostName {KATANA_ADDR}
    User {USER}
    IdentityFile ~/.ssh/id_rsa 
"""
    # Add katana to ssh config
    with open(SSH_CONFIG_PATH, "a") as f:
        f.write(appended)
 

Found katana in ssh config


In [11]:
print(appended)


Host asdf
    HostName katana.restech.unsw.edu.au
    User USERNAME_GOES_HERE
    IdentityFile ~/.ssh/id_rsa 



In [13]:
node_name = "k106"

USER = "z5207455" # TODO remove hardcoding
REMOTE_DIR = SCRATCH_DIR = Path(f"/srv/scratch/") / USER
NODE_NAME = node_name 
use_username = False
if use_username: 
    PORT = 22
    remote = f"{USER}@{KATANA_ADDR}:{PORT}"
else:
    remote = f"{KATANA}-{NODE_NAME}"

cmd = f"code --remote ssh-remote+{remote} {REMOTE_DIR}"
subprocess.run(cmd)

FileNotFoundError: [Errno 2] No such file or directory: 'code --remote ssh-remote+katana-k106 /srv/scratch/z5207455'

In [22]:
import shlex

In [25]:
# NOTE: for `shell=False` we need to supply args as a list

cmd = ["which", "code"]
captured = subprocess.run(cmd, capture_output=True, shell=False)
print(captured.stdout.decode())

/usr/local/bin/code



In [27]:
cmd = ["ssh", "katana", "echo 'hello'"]
captured = subprocess.run(cmd, capture_output=True, shell=False)
print(captured.stdout.decode())

hello



In [ ]:
# cmd = ["ssh", "-t", "katana", "sbf; ls"]

# TODO: don't use `-t` as it adds more output 


captured = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=False)
print(captured.stdout.read().decode())
print(captured.stderr.read().decode())

6275023.kman.restech.unsw.edu.au.OU
6275038.kman.restech.unsw.edu.au.OU
HYBRID
JOB_SUBS
install.sh
kamurani
miniconda3
peptide
sbf
test_script.sh

bash: sbf: command not found



In [ ]:

from functools import partial



class PBSServer:
    def __init__(
        self, 
        remotehost: str = "katana",
        print_output: bool = False,

    ): 
        self.remotehost = remotehost
        self.print_output = print_output

        #initialise the print_stdout decorator with print_output set to True

        # TODO: 
        # figure out how to add a decorator which, depending on a class attribute (self.print_output), 
        # will do different things.
        # self.print_stdout = partial(print_stdout, print_output=True)



    """Decorator for stdout collection"""
    def print_stdout(func):
        def decorated(self, *args, **kwargs):
            stdout, stderr = None, None
            try:
                stdout, stderr = func(self, *args, **kwargs)
            except Exception as e:
                print(e)
            
            if self.print_output: 
                print(stdout)
                print(stderr)
            return stdout, stderr

        return decorated
    
    
    # use partial to set print_output when initializing the class

    
        

    @print_stdout
    def ssh_execute(self, cmd):
        cmd = ["ssh", self.remotehost, cmd]
        captured = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=False)
        stdout = captured.stdout.read().decode()
        stderr = captured.stderr.read().decode()
        return stdout, stderr
    
    def qstat(self):
        pass 
    
    @print_stdout
    def pstat(self):
        """Get overview of the compute nodes and list of jobs running on each node."""
        cmd = "pstat"
        stdout, stderr = self.ssh_execute(cmd)
        return stdout, stderr  

    @print_stdout
    def pbsnodes(self, node: str):
        cmd = f"pbsnodes {node}"
        stdout, stderr = self.ssh_execute(cmd)
        return stdout, stderr  


katana = PBSServer(
    print_output=False,
)



In [94]:
out, err = katana.ssh_execute(cmd = "qstat -u $USER -s")

PBSServer.ssh_execute() missing 1 required positional argument: 'self'


In [85]:
out

"\nkman.restech.unsw.edu.au: \n                                                            Req'd  Req'd   Elap\nJob ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time\n--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----\n6278655.kman.r* z5207455 mwacgpu* interacti*    --    1   6   46gb 12:00 Q   -- \n    -- \n"

In [78]:
out, err = katana.ssh_execute(cmd = "qstat -u $USER -s -f -n -1 -t ")
# -f: full output
# -n: display node names
# -1: display node name on same line as job
# -t: expand output 
# -u: filter by USER 

print(out)
print(err)

In [67]:
job_id = 6277362

out, err = katana.ssh_execute(cmd = f"qstat -r {job_id} -n")
print(out)
print(err)


kman.restech.unsw.edu.au: 
                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
6277362.kman.r* z5207455 mwacgpu* interacti* 10573*   1   6   46gb 12:00 R 01:00
   k106/1*6




In [69]:
katana.pstat()

k001  normal-mrcbio           free          28/44   700/1006gb  6276858*28
k002  normal-mrcbio           job-busy      44/44   152/ 376gb  6252499*16 6252777*16 6277090[333]*12
k003  normal-mrcbio           free          26/44   176/ 376gb  6276704*10 6277017*16
k004  normal-mrcbio           job-busy      44/44    74/ 376gb  6252790*16 6277238*28
k005  normal-ccrc             free          17/32   179/ 186gb  6271466*2 6277090[316]*12 6277262*3
k006  normal-physics          job-busy      32/32   172/ 186gb  6276662*8 6277090[279]*12 6277090[286]*12
k007  normal-physics          free          28/32   152/ 186gb  6277090[272]*12 6277525*16
k008  normal-physics          free          28/32    88/ 186gb  6253055*16 6277090[351]*12
k009  normal-physics          free          12/32    24/ 186gb  6277090[267]*12
k010  normal-physics          job-busy      32/32   138/ 186gb  6277090[255]*12 6277468*8 6277547*12
k011  normal-physics          free          30/32   142/ 186gb  6253058*16 6277263

In [ ]:
# qstat commands 
"qstat -u $USER -s" 